In [30]:

"""
@author: sumanyumuku98
"""

import numpy as np
import matplotlib.pyplot as plt
import torch 
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
%matplotlib inline
from numpy import moveaxis
from skimage import io, transform
from torch.autograd import Variable
from PIL import Image
import os
import glob
import pandas as pd
import random
from torch.nn import BatchNorm1d

import plotly
# import bokeh
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device='cpu'
print(device)

cuda


In [31]:
# """
# COnverting the Multi-Class Problem into Binary-CLass

# C0: Undistracted driving
# C1-C9: Distracted driving
# """
# INFO_FILE = pd.read_csv("state-farm-distracted-driver-detection/driver_imgs_list.csv")
# INFO_FILE['Binary_labels'] = INFO_FILE.apply(lambda row: 0 if row.classname=='c0' else 1, axis=1 )
# NUM_CLASSES=2
# INFO_FILE.head()
# temp_dir='data/v1_cam1_no_split/'
NUM_CLASSES=10
CLASS_LABELS=['c' + str(i) for i in range(10)]
CLASSES_LIST=['Drive Safe', 'Text Right', 'Talk Right', 'Text Left', 'Talk Left',
              'Adjust Radio', 'Drink', 'Reach Behind', 'Hair & Makeup', 'Talk Passenger']



In [32]:
IMAGE_PREDIR = 'data/v2_cam1_cam2_ split_by_driver/'
CAMERA1='Camera 1'
CAMERA2='Camera 2'

In [33]:
# Pretrained Models
#######################################################
mobilenet = models.mobilenet_v2(pretrained=True)
for param in mobilenet.parameters():
    param.requires_grad=False
mobilenet.classifier[1] = torch.nn.Linear(1280,4096)
mobilenet.classifier.add_module('2',torch.nn.ReLU(inplace=True))
mobilenet.classifier.add_module('3',torch.nn.Linear(4096,4096))
mobilenet.classifier.add_module('4',torch.nn.ReLU(inplace=True))
mobilenet.classifier.add_module('5',torch.nn.Linear(4096,NUM_CLASSES))

# mobilenet.classifier.add_module("2",torch.nn.Softmax(dim=-1))
########################################################
resnet = models.resnet50(pretrained=True)
for param in resnet.parameters():
    param.requires_grad=False
resnet.fc = torch.nn.Linear(2048,4096)
resnetMod=torch.nn.Sequential(resnet,
                             torch.nn.ReLU(inplace=True),
                             torch.nn.Dropout(0.2,inplace=True),
                             torch.nn.Linear(4096,4096),
                             torch.nn.ReLU(inplace=True),
                             torch.nn.Linear(4096,NUM_CLASSES))



########################################################
vgg = models.vgg16(pretrained=True)
for param in vgg.parameters():
    param.requires_grad=False
vgg.classifier[6] = torch.nn.Linear(4096,4096)
vgg.classifier.add_module('7',BatchNorm1d(4096))
vgg.classifier.add_module('8',torch.nn.Linear(4096,2048))
vgg.classifier.add_module('9',torch.nn.ReLU(inplace=True))
# vgg.classifier.add_module('10',torch.nn.Dropout(0.3,inplace=True))
vgg.classifier.add_module('10',torch.nn.Linear(2048,2048))
vgg.classifier.add_module('11',torch.nn.ReLU(inplace=True))
vgg.classifier.add_module('12',torch.nn.Linear(2048,512))
vgg.classifier.add_module('13',torch.nn.ReLU(inplace=True))
vgg.classifier.add_module('14',torch.nn.Linear(512,NUM_CLASSES))

# vgg.classifier.add_module('11',torch.nn.)


# vgg.classifier.add_module("softmax",torch.nn.Softmax(dim=-1))

In [34]:
# print(resnetMod)

In [35]:
squeezenet = models.squeezenet1_1(pretrained=True)
# print(squeezenet)
for param in squeezenet.parameters():
    param.requires_grad=False
squeezenet.classifier[1]=torch.nn.Conv2d(512,1024,kernel_size=(1,1),stride=(1,1))
squeezenetMod=torch.nn.Sequential(squeezenet,
                                 torch.nn.Linear(1024,4096),
                                 torch.nn.ReLU(inplace=True),
                                 torch.nn.Linear(4096,4096),
                                 torch.nn.ReLU(inplace=True),
                                 torch.nn.Linear(4096,NUM_CLASSES))

    
#############################
mnasnet = models.mnasnet1_0(pretrained=True)
for param in mnasnet.parameters():
    param.requires_grad=False
mnasnet.classifier[1] = torch.nn.Linear(1280,4096)
mnasnet.classifier.add_module("2",torch.nn.ReLU(inplace=True))
mnasnet.classifier.add_module("3",torch.nn.Linear(4096,4096))
# mnasnet.classifier.add_module("4",torch.nn.Dropout(0.2,inplace=True))
mnasnet.classifier.add_module("4",torch.nn.ReLU(inplace=True))
mnasnet.classifier.add_module("5",torch.nn.Dropout(0.2,inplace=True))
mnasnet.classifier.add_module("6",torch.nn.Linear(4096,NUM_CLASSES))
# mnasnet.classifier.add_module("2",torch.nn.Softmax(dim=-1))

In [36]:
# for param in squeezenet.named_parameters():
#     if param[1].requires_grad:
#         print(param[0])

In [37]:
# print(vgg)

In [38]:
"""
Preprocess for pytorch pretrained models
"""

def preprocess(sample):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    transform=  transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,])
    
    img = sample['image']
    label = sample['label']
    
#     print(label, type(label))
    
    sample['image'] = transform(img)
    sample['label'] = torch.as_tensor(label)
    
    return sample

def preprocessVal(sample):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    transform=  transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,])
    
    img = sample['image']
    label = sample['label']
    
#     print(label, type(label))
    
    sample['image'] = transform(img)
    sample['label'] = torch.as_tensor(label)
    
    return sample

    
class Driver_Dataset(Dataset):
    
    def __init__(self,IMAGE_PREDIR,preprocess = None, mode='train',split=0.8):
        
        self.root_dir = IMAGE_PREDIR
        self.transform = preprocess
        self.mode=mode
        self.split=split
        self.totalTrainImages=[]
        self.totalTrainLabels=[]
        self.totalTestImages=[]
        self.totalTestLabels=[]
        self.images=None
        self.labels=None
        
        for c in CLASS_LABELS:
            for name in glob.glob(self.root_dir+CAMERA1+'/train/'+c+'/*.jpg'):
                self.totalTrainImages.append(name)
                self.totalTrainLabels.append(CLASS_LABELS.index(c))
            for name in glob.glob(self.root_dir+CAMERA2+'/train/'+c+'/*.jpg'):
                self.totalTrainImages.append(name)
                self.totalTrainLabels.append(CLASS_LABELS.index(c))
                
            for name in glob.glob(self.root_dir+CAMERA1+'/test/'+c+'/*.jpg'):
                self.totalTestImages.append(name)
                self.totalTestLabels.append(CLASS_LABELS.index(c))
            for name in glob.glob(self.root_dir+CAMERA2+'/test/'+c+'/*.jpg'):
                self.totalTestImages.append(name)
                self.totalTestLabels.append(CLASS_LABELS.index(c))
        
        shuffle_list=list(zip(self.totalTrainImages,self.totalTrainLabels))
        random.shuffle(shuffle_list)
        self.totalTrainImages,self.totalTrainLabels=zip(*shuffle_list)
        
        if self.mode=='train':
            length=len(self.totalTrainImages)
            self.images= self.totalTrainImages[:int(self.split*length)]
            self.labels=self.totalTrainLabels[:int(self.split*length)]
        
        elif self.mode=='val':
            length=len(self.totalTrainImages)
            self.images= self.totalTrainImages[int(self.split*length):]
            self.labels=self.totalTrainLabels[int(self.split*length):]
        else:
            self.images=self.totalTestImages
            self.labels=self.totalTestLabels

        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        
        label = self.labels[idx]
#         print(type(label))
        img_name = self.images[idx]
        img = Image.open(img_name)

        sample = {'image': img, 'label': label}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    def listcheck(self):
        return self.images,self.labels
      


In [39]:
trainData= Driver_Dataset(IMAGE_PREDIR,preprocess,'train',0.95)
valData=Driver_Dataset(IMAGE_PREDIR,preprocessVal,'val',0.95)
trainLoader=DataLoader(trainData,batch_size=16,num_workers=2)
valLoader=DataLoader(valData,batch_size=16,num_workers=2)
# DataLoader?

In [40]:
Crossloss=torch.nn.CrossEntropyLoss()


In [41]:
def train(model,dataloaderList,epochs, device = 'cpu',saveName=None):
    model.to(device)
    trainloader=dataloaderList[0]
    valloader=dataloaderList[1]
    paramsToUpdate=[]
    
    for param in model.parameters():
        if param.requires_grad:
            paramsToUpdate.append(param)
    
    optimizer=torch.optim.SGD(paramsToUpdate,lr=1e-3)

    valAccList=[]
    
    print('Start Training...')
    for epoch in range(epochs):
        model.train()
        running_loss=0
        
        for i_batch, data in enumerate(trainloader):
            

            images=Variable(data['image'],requires_grad=True).to(device)
            labels=Variable(data['label']).to(device)
            
            outputs=model(images)
            
            optimizer.zero_grad()

            loss=Crossloss(outputs,labels)
            loss.backward()
            optimizer.step()
            
            running_loss+=loss.item()
            
            # Print Training Statistics
            del images,labels,outputs
            
            if i_batch%100==99:
                print('[%2d,%5d] Training Loss: %.3f'  % 
                      (epoch+1,i_batch+1,running_loss/100))
                running_loss=0
                
        print('Validating the model')
        
        with torch.no_grad():
            model.eval()
            valPreds=[]
            valLabels=[]
            for i_batch, data in enumerate(valloader):
                
                images= data['image'].to(device)
                labels= data['label'].numpy()
                
                preds=model(images)
                preds=preds.cpu().numpy()
                
                pred_labels=np.argmax(preds,axis=-1)
                valPreds.append(pred_labels)
                valLabels.append(labels)
            
            valPreds=np.concatenate(valPreds)
            valLabels=np.concatenate(valLabels)
            size=valPreds.shape[0]
            running_corrects=np.sum(valPreds==valLabels)
        
            acc=float(running_corrects)/size
            valAccList.append(acc)
            print('Accuracy after epoch %2d is %.3f' % (epoch+1,acc))
#             print(valPreds)
#             print(valLabels)
    
    print("Finished Training")
    print("Saving Model")
    torch.save(model.state_dict(),'Model_weights/'+saveName+'.pt')
    
    return model,valAccList
    

In [42]:
trainedModel,valAcc=  train(vgg,[trainLoader,valLoader],2,device,'vgg')

Start Training...
[ 1,  100] Training Loss: 2.292
[ 1,  200] Training Loss: 2.285
[ 1,  300] Training Loss: 2.275
[ 1,  400] Training Loss: 2.272
[ 1,  500] Training Loss: 2.259
[ 1,  600] Training Loss: 2.256
[ 1,  700] Training Loss: 2.256
Validating the model
Accuracy after epoch  1 is 0.236
[ 2,  100] Training Loss: 2.237
[ 2,  200] Training Loss: 2.242
[ 2,  300] Training Loss: 2.234
[ 2,  400] Training Loss: 2.237
[ 2,  500] Training Loss: 2.226
[ 2,  600] Training Loss: 2.229
[ 2,  700] Training Loss: 2.239
Validating the model
Accuracy after epoch  2 is 0.236
Finished Training
Saving Model
